# P4_Florian_Cazac
## Transportation problem

1. Read an external file containing the parameters of a transportation problem LP problem
2. Different methods can be used to find the initial feasible solution for the problem:
    * Northwest corner rule
    * Minimum cost method
    * Minimum Row cost method
    * Vogel's method
3. the transportation simplex algorithm in an arbitrary transportation problem. 

In [91]:
import pandas as pd
import numpy as np

## 1. Read a file containing the parameters of a transportation problem LP problem

In [92]:
filename = 'transportation_problem.csv'

In [93]:
data = pd.read_csv(filename, index_col=0)
print(data)

supply = data['Supply'][:-1].to_numpy()
demand = data.loc["Demand"].drop("Supply").to_numpy()
cost = data.iloc[:-1, :-1].to_numpy()

print("Supply:", supply)
print("Demand:", demand)
print("Cost:", cost)


         D1   D2   D3   D4  Supply
\                                 
S1       12   13    4    6   500.0
S2        6    4   10   11   700.0
S3       10    9   12    4   800.0
Demand  400  900  200  500     NaN
Supply: [500. 700. 800.]
Demand: [400. 900. 200. 500.]
Cost: [[12 13  4  6]
 [ 6  4 10 11]
 [10  9 12  4]]


## 2. Different methods can be used to find the initial feasible solution

### 1. Northwest corner rule

In [94]:
# North West Corner Method

def north_west_corner(supply, demand):
    supply = supply.copy()
    demand = demand.copy()
    i = 0
    j = 0
    allocation = np.zeros((supply.size, demand.size))
    while i < supply.size and j < demand.size:
        if supply[i] < demand[j]:
            allocation[i][j] = supply[i]
            demand[j] -= supply[i]
            i += 1
        else:
            allocation[i][j] = demand[j]
            supply[i] -= demand[j]
            j += 1
    return allocation

def get_cost(allocation, cost):
    return np.sum(allocation * cost)

In [95]:
north_west_corner_allocation = north_west_corner(supply, demand)
print("NorthWest Corner Method")
print(north_west_corner_allocation)
print("\nCost: ", get_cost(north_west_corner_allocation, cost))

NorthWest Corner Method
[[400. 100.   0.   0.]
 [  0. 700.   0.   0.]
 [  0. 100. 200. 500.]]

Cost:  14200.0


### 2. Minimum cost method

In [96]:
# Minimum Cost Method

def minimum_cost_method(supply, demand, costs):
    supply = supply.copy()
    demand = demand.copy()
    allocation = np.zeros_like(costs, dtype=float)  # Allocation matrix
    
    # Get the indices of costs in ascending order
    cost_indices = np.dstack(np.unravel_index(np.argsort(costs, axis=None), costs.shape))[0]
    
    for i, j in cost_indices:
        if supply[i] == 0 or demand[j] == 0:  # Skip exhausted rows/columns
            continue
        
        # Allocate as much as possible to the minimum cost cell
        allocated = min(supply[i], demand[j])
        allocation[i, j] = allocated
        supply[i] -= allocated
        demand[j] -= allocated
    
    return allocation


In [97]:
minimum_cost_allocation = minimum_cost_method(supply, demand, cost)
print("\nMinimum Cost Method")
print(minimum_cost_allocation)
print("\nCost: ", get_cost(minimum_cost_allocation, cost))


Minimum Cost Method
[[300.   0. 200.   0.]
 [  0. 700.   0.   0.]
 [100. 200.   0. 500.]]

Cost:  12000.0


### 3. Minimum Row cost method

In [98]:
def minimum_row_cost_method(supply, demand, cost_matrix):
    """
    Solves the transportation problem using the Minimum Row Cost Method.
    
    Args:
    cost_matrix (list of lists): The cost matrix (rows: suppliers, columns: destinations).
    supply (list): Available supply at each source.
    demand (list): Required demand at each destination.
    
    Returns:
    allocation (2D list): Initial feasible allocation matrix.
    """
    cost_matrix = np.array(cost_matrix, dtype=float)
    supply = supply.copy()  # Avoid modifying the original list
    demand = demand.copy()  # Avoid modifying the original list
    rows, cols = cost_matrix.shape
    allocation = np.zeros((rows, cols), dtype=float)
    
    for i in range(rows):
        while supply[i] > 0:  # While there is supply at row i
            # Find the column with the minimum cost in the current row
            min_cost_index = np.argmin(cost_matrix[i, :])
            
            # Allocate as much as possible to the selected cell
            allocation_quantity = min(supply[i], demand[min_cost_index])
            allocation[i, min_cost_index] = allocation_quantity
            
            # Update supply and demand
            supply[i] -= allocation_quantity
            demand[min_cost_index] -= allocation_quantity
            
            # If demand is met, set the cost to infinity for that column
            if demand[min_cost_index] == 0:
                cost_matrix[:, min_cost_index] = np.inf

    return allocation

In [99]:
minimum_row_cost_allocation = minimum_row_cost_method(supply, demand, cost)
print("\nMinimum Row Cost Method")
print(minimum_row_cost_allocation)
print("\nCost: ", get_cost(minimum_row_cost_allocation, cost))


Minimum Row Cost Method
[[  0.   0. 200. 300.]
 [  0. 700.   0.   0.]
 [400. 200.   0. 200.]]

Cost:  12000.0


### 4. Vogel's method

In [100]:
def calculate_row_penalties(costs, supply):
    """Calculate penalties for each row."""
    supply = supply.copy()
    row_penalties = np.zeros(costs.shape[0])
    for i in range(costs.shape[0]):
        if supply[i] > 0:
            row = costs[i, :]
            sorted_row = np.sort(row)
            if len(sorted_row) > 1:
                row_penalties[i] = sorted_row[1] - sorted_row[0]
            else:
                row_penalties[i] = float('inf')
    return row_penalties

def calculate_column_penalties(costs, demand):
    demand = demand.copy()
    """Calculate penalties for each column."""
    col_penalties = np.zeros(costs.shape[1])
    for j in range(costs.shape[1]):
        if demand[j] > 0:
            col = costs[:, j]
            sorted_col = np.sort(col)
            if len(sorted_col) > 1:
                col_penalties[j] = sorted_col[1] - sorted_col[0]
            else:
                col_penalties[j] = float('inf')
    return col_penalties

In [101]:
def vogels_approximation_method(supply, demand, costs):
    # Convert the costs matrix to float to handle np.inf properly
    supply = supply.copy()
    demand = demand.copy()
    costs = costs.astype(float)
    
    # Initialize the allocation matrix with zeros
    allocation = np.zeros_like(costs, dtype=float)
    
    while np.any(supply > 0) and np.any(demand > 0):
        # Step 1: Calculate penalties for rows and columns
        row_penalties = calculate_row_penalties(costs, supply)
        col_penalties = calculate_column_penalties(costs, demand)
        
        # Step 2: Find the highest penalty and the corresponding row/column
        if np.max(row_penalties) >= np.max(col_penalties):
            # Find the row with the maximum penalty
            row_index = np.argmax(row_penalties)
            col_index = np.argmin(costs[row_index, :])  # Find the column with the minimum cost
        else:
            # Find the column with the maximum penalty
            col_index = np.argmax(col_penalties)
            row_index = np.argmin(costs[:, col_index])  # Find the row with the minimum cost
        
        # Step 3: Allocate as much as possible to the cell
        allocation_amount = min(supply[row_index], demand[col_index])
        allocation[row_index, col_index] = allocation_amount
        supply[row_index] -= allocation_amount
        demand[col_index] -= allocation_amount
        
        # Step 4: If demand or supply is exhausted, set corresponding row/column to np.inf
        if supply[row_index] == 0:
            costs[row_index, :] = np.inf  # Set row to inf
        if demand[col_index] == 0:
            costs[:, col_index] = np.inf  # Set column to inf
    
    return allocation

In [102]:
row_penalities = calculate_row_penalties(cost, supply)
col_penalities = calculate_column_penalties(cost, demand)
print("\nRow Penalties")
print(row_penalities)
print("\nColumn Penalties")
print(col_penalities)

vogels_allocation = vogels_approximation_method(supply, demand, cost)
print("\nVogel's Approximation Method")
print(vogels_allocation)

print("\nCost: ", get_cost(vogels_allocation, cost))


Row Penalties
[2. 2. 5.]

Column Penalties
[4. 5. 6. 2.]

Vogel's Approximation Method
[[  0.   0. 200. 300.]
 [  0. 700.   0.   0.]
 [400. 200.   0. 200.]]

Cost:  12000.0


## 3. Transportation Simplex

In [103]:
print(supply)
print(demand)
print(cost)

[500. 700. 800.]
[400. 900. 200. 500.]
[[12 13  4  6]
 [ 6  4 10 11]
 [10  9 12  4]]


In [104]:
# Step 1 - Create helper functions for the transportation simplex method

import numpy as np

# Function to compute the u and v values (dual variables)
def get_us_and_vs(bfs, costs):
    us = [None] * len(costs)
    vs = [None] * len(costs[0])
    us[0] = 0
    bfs_copy = bfs.copy()
    
    while len(bfs_copy) > 0:
        for index, bv in enumerate(bfs_copy):
            i, j = bv[0]
            if us[i] is None and vs[j] is None: 
                continue
            cost = costs[i][j]
            if us[i] is None:
                us[i] = cost - vs[j]
            else:
                vs[j] = cost - us[i]
            bfs_copy.pop(index)
            break
    
    return us, vs

# Function to compute the reduced costs (opportunity costs) for non-basic variables
def get_ws(bfs, costs, us, vs):
    ws = []
    for i, row in enumerate(costs):
        for j, cost in enumerate(row):
            non_basic = all([p[0] != i or p[1] != j for p, v in bfs])
            if non_basic:
                ws.append(((i, j), us[i] + vs[j] - cost))
    
    return ws

# Check if there is any possibility of improving the solution
def can_be_improved(ws):
    for p, v in ws:
        if v > 0: return True
    return False

# Get the position of the entering variable with the most positive reduced cost
def get_entering_variable_position(ws):
    ws_copy = ws.copy()
    ws_copy.sort(key=lambda w: w[1])
    return ws_copy[-1][0]

# Helper function to find the possible next nodes in the loop
def get_possible_next_nodes(loop, not_visited):
    last_node = loop[-1]
    nodes_in_row = [n for n in not_visited if n[0] == last_node[0]]
    nodes_in_column = [n for n in not_visited if n[1] == last_node[1]]
    if len(loop) < 2:
        return nodes_in_row + nodes_in_column
    else:
        prev_node = loop[-2]
        row_move = prev_node[0] == last_node[0]
        if row_move: return nodes_in_column
        return nodes_in_row


def loop_pivoting(bfs, loop):
    even_cells = loop[0::2]
    odd_cells = loop[1::2]
    get_bv = lambda pos: next(v for p, v in bfs if p == pos)
    leaving_position = sorted(odd_cells, key=get_bv)[0]
    leaving_value = get_bv(leaving_position)
    
    new_bfs = []
    for p, v in [bv for bv in bfs if bv[0] != leaving_position] + [(loop[0], 0)]:
        if p in even_cells:
            v += leaving_value
        elif p in odd_cells:
            v -= leaving_value
        new_bfs.append((p, v))
        
    return new_bfs
    


# Find the loop to apply pivoting (cycling through the basic variables)
def get_loop(bv_positions, ev_position):
    def inner(loop):
        if len(loop) > 3:
            can_be_closed = len(get_possible_next_nodes(loop, [ev_position])) == 1
            if can_be_closed: return loop
        
        not_visited = list(set(bv_positions) - set(loop))
        possible_next_nodes = get_possible_next_nodes(loop, not_visited)
        for next_node in possible_next_nodes:
            new_loop = inner(loop + [next_node])
            if new_loop: return new_loop
    
    return inner([ev_position])

# Step 2 - Implement the transportation simplex method
def transportation_simplex_method(supply, demand, costs, initial_allocation):
    # Helper function to get the basic feasible solution (from an initial allocation)
    def get_basic_variables(initial_allocation):
        bfs = []
        for i in range(len(initial_allocation)):
            for j in range(len(initial_allocation[i])):
                if initial_allocation[i, j] > 0:
                    bfs.append(((i, j), initial_allocation[i, j]))
        return bfs

    # Extract the basic variables (initial allocation)
    bfs = get_basic_variables(initial_allocation)
    
    # Start the simplex algorithm
    def inner(bfs):
        us, vs = get_us_and_vs(bfs, costs)
        ws = get_ws(bfs, costs, us, vs)
        
        # If there is an opportunity to improve the solution
        if can_be_improved(ws):
            ev_position = get_entering_variable_position(ws)
            loop = get_loop([p for p, v in bfs], ev_position)
            return inner(loop_pivoting(bfs, loop))  # Apply the pivoting to update BFS
        return bfs

    # Perform the transportation simplex method
    bfs_optimized = inner(bfs)
    
    # Construct the optimized solution
    optimized_allocation = np.zeros_like(costs)
    for (i, j), v in bfs_optimized:
        optimized_allocation[i][j] = v
    
    # Calculate the optimized total cost
    optimized_cost = np.sum(optimized_allocation * costs)
    
    return optimized_allocation, optimized_cost

In [105]:
# Step 3 - Apply to your existing method (use the NWCR, MCM, MRCM, or VAM allocation)

NWRCallocation =  north_west_corner_allocation
MCMallocation = minimum_cost_allocation
MRCMallocation = minimum_row_cost_allocation
VAMallocation = vogels_allocation

# Apply the transportation simplex method to the initial allocations
optimized_NWRC, optimized_cost_NWRC = transportation_simplex_method(supply, demand, cost, NWRCallocation)
optimized_MCM, optimized_cost_MCM = transportation_simplex_method(supply, demand, cost, MCMallocation)
optimized_MRCM, optimized_cost_MRCM = transportation_simplex_method(supply, demand, cost, MRCMallocation)
optimized_VAM, optimized_cost_VAM = transportation_simplex_method(supply, demand, cost, VAMallocation)

print("\nOptimized NWRC Allocation")
print(optimized_NWRC)
print("\nCost: ", optimized_cost_NWRC)

print("\nOptimized MCM Allocation")
print(optimized_MCM)
print("\nCost: ", optimized_cost_MCM)

print("\nOptimized MRCM Allocation")
print(optimized_MRCM)
print("\nCost: ", optimized_cost_MRCM)

print("\nOptimized VAM Allocation")
print(optimized_VAM)
print("\nCost: ", optimized_cost_VAM)



Optimized NWRC Allocation
[[300   0 200   0]
 [  0 700   0   0]
 [100 200   0 500]]

Cost:  12000

Optimized MCM Allocation
[[300   0 200   0]
 [  0 700   0   0]
 [100 200   0 500]]

Cost:  12000

Optimized MRCM Allocation
[[  0   0 200 300]
 [  0 700   0   0]
 [400 200   0 200]]

Cost:  12000

Optimized VAM Allocation
[[  0   0 200 300]
 [  0 700   0   0]
 [400 200   0 200]]

Cost:  12000
